Importing the Library

In [1]:
import cv2
import os # To find the training Files in the path
import numpy as np 

Function for Detecting Face using Frontal face Haadcascade

In [2]:
def front_face_Detection(image):
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # this converts the Color image to Gray image for better classifying
    face_cascade = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
    # face cascade object is formed to find corrdinates of faces
    face_coordinates = face_cascade.detectMultiScale(gray_image, scaleFactor = 1.05, minNeighbors = 5)
    # detecting the face corrdinates by giving the attribute (images, scale factor by what percentage should the image decrease everytime, minimum neighbors for classification)
    return face_coordinates, gray_image

Function for giving unique id to each training images and  accesing it from System Directory 

In [3]:
def labels_to_images(directory):
    face_coordinates = []
    # creating list to store face_coordinates for each training face 
    face_id = []
    # creating a list to store face id for eachtraining images
    for (root, subdirectory, files) in os.walk(directory): 
    # the os walks through root, sub-directory, and each files in the system 
        for each_file in files: 
        #for each file in the files
            if each_file.startswith("."):
            # if some external file startswith .extension eliminate that file
                print("Image can't be read, skipping the file")
                continue
            unique_id = os.path.basename(root)
            # in the directory each image subdirectory should be saved as integer names, because it takes only integer input
            #each unique_id for each_image
            image_path = os.path.join(root, each_file)
            # the path for each image
            train_image = cv2.imread(image_path)
            coordinates, gray_image = front_face_Detection(train_image)
            if len(coordinates) != 1 :
            # if images contain more than 1 image eliminate that image
                #print("Image cant be read as for training only 1 image is required"), #for debugging
               # print("Id = ", unique_id), #for debugging
                continue
            x, y, w, h = coordinates[0]
            #getting coordinates of face
            test_image = gray_image[y:y+h, x:x+w]
            # taking only the image of the cropped faces  
            face_coordinates.append(test_image) # storing the values in list of image in face Coordinates
            face_id.append(int(unique_id)) # storing only the integer values of face id 
    return face_coordinates, face_id

Function for training each image using LBPH Face Recognizer Algorithm

In [4]:
def training_images( face_coordinates, face_id):
    face_recognizer = cv2.face.LBPHFaceRecognizer_create()
    # creating a LBPH Face_Recognizer Object
    face_recognizer.train(face_coordinates, np.array(face_id))
    #it trains the image and takes attributes as face_coordinates and n-dimensional array  face_id
    return face_recognizer
#Local Binary Pattern Histogram (LBPH) , generally it takes 50k images to give a perfect result
#we are using here around 30 images

Drawing rectangle over the face 

In [5]:
def draw_rectangle(image, face_coordinates):
    x, y, w, h = face_coordinates
    # x, y - coordinates and w,h are the width and height of the images
    cv2.rectangle(image, (x,y), (x + w, y + h), (0, 0, 255), 2)
    # (the attributes too draw rectangle are (image, (the starting coordinates), (diagonally opposite coordinates), (color of border), width of border))

Function To write predicted name on the image 

In [6]:
def write_text(image, text_name, x, y):
    cv2.putText(image, text_name, (x, y), cv2.FONT_HERSHEY_SIMPLEX, 0.7,  (152, 251, 152), 1)
    # attributes to write text are (image, text_name, (coordinates where to be written), FOnt style, scale of letter, color of letter, width of letter)

We have made every Function now, we have to feed  training data to the code and predict the test images  

In [7]:
face_coordinates, face_id = labels_to_images("D:\study\OpenCV\Train_images_1")

In [8]:
face_trained = training_images(face_coordinates, face_id)
face_trained.save("Trained_images.yml")
#this is to save the trained images, so that we dont need to run files again and again

Testing the Image

In [9]:
testing_image = cv2.imread("test1.jpg")

In [10]:
face_detected, gray_image = front_face_Detection(testing_image)

In [11]:
Names = { 0:"Bill_Gates", 1:"Elon_Musk", 2:"Mark_Zuckerberg", 3:"Warren_Buffet", 4:"Zeff_Bezos", 5:"Ishan"}
#assigning the names to the concurrent _id of the person

In [12]:
for face in face_detected:
    x, y, w, h = face
    # assigning the coordinates of faces to the variables
    cropped_face_image = gray_image[y:y+h, x:x+w]
    #predicting only the cropped face
    label, confidence = face_trained.predict(cropped_face_image)
    #label is the image_id for concurrent person face
    #confidence value shows something of accuracy, if confidence value is less than or equal to 37 than it shows perfect match.
    predicted_name = Names[label]
    #predicted the name for consequent image
    draw_rectangle(testing_image, face)
    #drawing rectangle to thepredicted face
    write_text(testing_image, predicted_name, x, y)
    #writing the concurrent names to the concurrent face
cv2.imshow("Image", testing_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

Recognizing Face using WebCam

In [13]:
webcam = cv2.VideoCapture(1)
webcam.set(3, 700)
webcam.set(4, 300)
while True:
    check, frame = webcam.read()
    #cv2.imshow("Video", frame)
    face_detected, gray_face = front_face_Detection(frame)
    for face in face_detected:
        x, y, w, h = face
        cropped_face_image = gray_face[y:y+h, x:x+w]
        label, confidence = face_trained.predict(cropped_face_image)
        predicted_name = Names[label]
        draw_rectangle(frame, face)
        write_text(frame, predicted_name, x, y)
    
    cv2.imshow("Video", frame)
    key = cv2.waitKey(1)
    if key == ord('a'):
        break
webcam.release()
cv2.destroyAllWindows()